## The ESPN fantasy league assigns value to some recorded player statistics. The values include:

- Points scored PTS = 1
- Blocks BLK = 4
- Assists AST = 2
- Rebounds REB = 1
- Field goals made FGM = 2
- Free throws made FTM = 1
- Three pointers made PM3 = 1
- Steals STL = 4

## Some values result in a penalty (negative scores):

- Turnovers TOV = -2
- Field goals attempted FGA = -1
- Free throws attempted FTA = -1

The f

In [1]:
stats = {"PTS":1,"BLK":4,"AST":2,"REB":1,"TOV":-2,"FGM":2,"FGA":-1,"FTM":1,"FTA":-1,"PM3":1,"STL":4}

In [2]:
stats.values()

dict_values([1, 4, 2, 1, -2, 2, -1, 1, -1, 1, 4])

### The 

In [3]:
import numpy as np

In [4]:
scores = np.array(list(stats.values()))

In [5]:
scores.dtype

dtype('int32')

In [6]:
fs = ", ".join(list(stats.keys()))

In [7]:
fs

'PTS, BLK, AST, REB, TOV, FGM, FGA, FTM, FTA, PM3, STL'

In [8]:
import sqlalchemy
import pandas as pd
from os import environ
from time import localtime

engine = sqlalchemy.create_engine("mariadb+mariadbconnector://"+environ.get("USER")+\
                                  ":"+environ.get("PSWD")+"@127.0.0.1:3306/nba")

In [9]:
fields = "Name, " + fs + ", Game_day "

join =  "Box_scores INNER JOIN Players on Box_scores.Player_ID = Players.ID "

players = ['Eric Bledsoe', 'Chris Boucher', 'P.J. Washington', 'Brandon Clarke','Patrick Beverley',
          "De'Anthony Melton", 'Coby White', 'Onyeka Okongwu', 'Chris Duarte', 'Jordan Clarkson',
           'Richaun Holmes']
condition = "where Name IN " + str(tuple(players))

# select = "SELECT "+ fields + " FROM " + join + condition + "ORDER BY Game_day desc"
select = "SELECT "+ fields + " FROM " + join + "ORDER BY Game_day desc"

In [10]:
boxes = pd.read_sql(select,engine)

In [11]:
boxes["Game_day"] = pd.to_datetime(boxes["Game_day"])

In [12]:
boxes

,Name,PTS,BLK,AST,REB,TOV,FGM,FGA,FTM,FTA,PM3,STL,Game_day
0,Udoka Azubuike,7,3,0,14,1,3,5,1,2,0,0,2022-02-07
1,Jordan Clarkson,16,0,2,2,1,6,12,1,1,3,0,2022-02-07
2,Nerlens Noel,0,0,0,3,1,0,1,0,2,0,1,2022-02-07
3,Mike Conley,18,1,7,3,1,5,17,6,6,2,0,2022-02-07
4,Jared Butler,0,0,0,0,1,0,0,0,0,0,0,2022-02-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17273,Kevon Looney,7,1,1,4,1,3,5,1,2,0,1,2021-10-19
17274,Draymond Green,6,0,6,8,5,2,5,2,4,0,1,2021-10-19
17275,Carmelo Anthony,9,0,2,4,1,3,9,1,2,2,0,2021-10-19
17276,Kent Bazemore,8,1,0,2,1,3,9,0,0,2,0,2021-10-19


In [13]:
def calc_fantasy_points(scores, boxes):
    return scores @ boxes

In [14]:
a = np.array(boxes.values[:,1:-1],dtype = np.int64)

In [16]:
fantasy_points = calc_fantasy_points(scores, a.T)

In [17]:
boxes["Fantasy Points"] = fantasy_points

In [18]:
names = boxes["Name"].unique()

In [19]:
rolls= boxes.sort_values(["Game_day"], ascending=[True]).groupby("Name").rolling(5,min_periods =1).mean()
# .rolling(5,min_periods =5).mean()
# df.dtypes

In [20]:
boxes["Rolling"] = 1

In [21]:
for i,j in rolls.index:
    boxes.loc[j,"Rolling"] = rolls.loc[i,j]["Fantasy Points"]

In [22]:
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import dcc

/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [23]:
def options(names):
    options = []
    for i in names:
        d = {}
        d["label"] = i
        d["value"] = i
        options.append(d)
        
    return options

In [24]:
def drpdwn():
    names = list(boxes["Name"].unique())
    opts = options(names)
    d = dcc.Dropdown(options = opts, value = names[0:2],id = 'dp',multi = True)
    
    return d

In [35]:
fig = px.line(y="Rolling", x="Game_day",color="Name",data_frame = boxes, markers=True,range_y=(0,100))
fig.update_layout(title ="Fantasy points over time",
                  title_x = 0.5,yaxis_title = "5 game Rolling mean / Fantasy points")


app = JupyterDash(__name__)
colours = {'text': '#7FDBFF', 'background':'#333333','radio_button':'#BBBBBB'} 
text_size = {'H1':48,'H2':40,'text':28,'radio_button':20}

app.layout = html.Div(style={'backgroundColor':colours['background'],'fontFamily':'Arial'}, children=[
    html.H1(children="ESPN Fantasy anlysis",
        style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['H1']}),
    
#     year_slider(),
    
    
    
    html.Div(children=[drpdwn(), dcc.Graph(figure = fig, id = 'graph')])



])

@app.callback(
    Output('graph','figure'),
    Input('dp','value'))
def update_figure(selected):
    d = boxes.loc[boxes['Name'].isin(selected)]
    
#     _ , percentages = get_highs_and_percentages(d, stats,len(d))
    fig = px.line(y="Rolling", x="Game_day",color="Name",data_frame = d, markers=True,range_y=(0,100))
    
#     fig.update_layout(title ="Fantasy points over time",
#                   title_x = 0.5)
    return fig

In [36]:
app.run_server(mode = "external")

/home/blunt/.local/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/
